<a href="https://colab.research.google.com/github/100jy/Dacon_card/blob/main/%EC%8B%A0%EC%9A%A9%EC%B9%B4%EB%93%9C_%EC%82%AC%EC%9A%A9%EC%9E%90_%EC%97%B0%EC%B2%B4_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import os
os.chdir('/content/drive/MyDrive/데이콘')

In [4]:
train = pd.read_csv('train.csv', index_col=0)
train = train.fillna('NA')

# 변수확인

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

for i in range(train.shape[1]):
  plt.figure()
  try:
    sns.kdeplot(train.iloc[:,i])
  except:
    plt.hist(train.iloc[:,i])
  plt.title(train.columns[i])

# 전처리

In [7]:
# 범주형 인코딩
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder

train = pd.read_csv('train.csv', index_col=0)
train = train.fillna('NA')

sub = pd.read_csv('sample_submission.csv')
test  = pd.read_csv('test.csv', index_col=0)
test = test.fillna('NA')

for cate in ['gender', 'car', 'reality', 'income_type',
       'edu_type', 'family_type', 'house_type', 
     'occyp_type',
     'family_size', 'begin_month']:
  inco = LabelEncoder() 
  inco.fit(train[cate])
  train[cate] = inco.transform(train[cate])
  test[cate] = inco.transform(test[cate])


X = train.iloc[:,:-1]
Y = train.iloc[:,[-1]].values
Y = Y.ravel()

# 모든 변수 minmax
scaler  = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)
test = scaler.transform(test)

# grid search

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split

# 오히려 과적합 된는데??????
# min_samples_split 작을 수록 max_depth 높을 수록 과적합 되기 쉬움
paras = {'n_estimators' : [10],
         'max_depth' : [12],
         'min_samples_split' : [8],
         'min_samples_leaf' : [8],
         }

clf = RandomForestClassifier()
cv = GridSearchCV(clf, param_grid=paras, cv=3)
cv.fit(X, Y)

In [15]:
print('최적 파라미터 : {}'.format(cv.best_params_))
print('최적 점수 : {}'.format(cv.best_score_))

최적 파라미터 : {'max_depth': 18, 'min_samples_leaf': 2, 'min_samples_split': 8, 'n_estimators': 10}
최적 점수 : 0.7017046528328986


# 모델링

In [20]:
from sklearn.metrics import log_loss, f1_score
from keras.utils import to_categorical


# StratifiedKfold(3)
# train val subsampling하여 여러 모델 피팅 후 앙상블
fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
total = 0
re = np.zeros_like(sub.iloc[:,1:].values, dtype=np.float32)

for train_idx, val_idx in fold.split(X, Y):
    train_x, val_x, train_y, val_y =  X[train_idx], X[val_idx],\
                                      Y[train_idx], Y[val_idx]
    # 랜포
    clf = RandomForestClassifier(**cv.best_params_)
    clf.fit(train_x, train_y)
    prob = clf.predict_proba(val_x)
    loss = log_loss(to_categorical(val_y), prob)
    re += clf.predict_proba(test)
    total += loss
    print('val_loss : {:.3f}'.format(loss))

print('mean_loss : {:.3f}'.format(total/5))

val_loss : 0.786
val_loss : 0.784
val_loss : 0.779
val_loss : 0.789
val_loss : 0.778
mean_loss : 0.783


In [21]:
sub.iloc[:,1:] = re / 5

In [22]:
sub.to_csv('baseline_submission.csv', index=False)

In [23]:
sub

,index,0,1,2
0,26457,0.103628,0.152585,0.743787
1,26458,0.153958,0.180243,0.665800
2,26459,0.093266,0.155461,0.751273
3,26460,0.138642,0.155834,0.705524
4,26461,0.128066,0.190785,0.681149
...,...,...,...,...
9995,36452,0.099212,0.208711,0.692077
9996,36453,0.111511,0.229944,0.658545
9997,36454,0.082081,0.156835,0.761083
9998,36455,0.115176,0.186276,0.698548
